<a href="https://colab.research.google.com/github/vinicius-souza-lima/mvp_ADBP/blob/main/MVP_ADBP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introdução


Nome:Vinícius de Souza Lima.

Link para o Reposítório: [MVP Análise de Dados e Boas Práticas](https://github.com/vinicius-souza-lima/mvp_ADBP)


Foi utilizado um dataset proveniente do Kaggle, [Steam Games Dataset](https://www.kaggle.com/datasets/fronkongames/steam-games-dataset?resource=download), o qual traz informações de mais de 85.000 jogos publicados na Steam, a maior plataforma de jogos de PC do mercado.

In [1]:
import pandas as pd
import matplotlib as plt

In [12]:
# Upload dos dados( Dataset proveniente do )
url_dados = 'https://media.githubusercontent.com/media/vinicius-souza-lima/mvp_ADBP/main/games.csv'

# carga do dataset através da url
games = pd.read_csv(url_dados)

In [ ]:
# Dicionário de Dados
colunas = games.columns
descriptions =
"""Identificador Único,string
Nome do Jogo,string
Data de Lançamento,string
Estimativa de Possuidores,string
Número de Usuários Atuais,int
Classificação indicativa,int
Preço em Dólares Americanos,float
Número de DLCs,int
Descrição detalhada dos jogos,string
Descrição curta dos jogos,string
Enumeração separada por vírgula das línguas suportadas, lista de strings
Enumeração separada por vírgula das línguas com suporte de aúdio, lista de strings
Imagem Principal na Loja,string
Website do Jogo,string
URL do Suporte do Jogo,string
Suporta Windows?,bool
Suporta Mac?,bool
Suporta Linux?,bool
Score do Metacritic,int
URL da review do Metecritic,string

"""

# Definição do Problema

# Análise de Dados

# Pré-Processamento de Dados

In [22]:
# exibindo as últimas linhas
games.describe()

,AppID,Peak CCU,Required age,Price,DLC count,Metacritic score,User score,Positive,Negative,Score rank,Achievements,Recommendations,Average playtime forever,Average playtime two weeks,Median playtime forever,Median playtime two weeks
count,8.510300e+04,85103.000000,85103.000000,85103.000000,85103.000000,85103.000000,85103.000000,8.510300e+04,85103.000000,44.000000,85103.000000,8.510300e+04,85103.000000,85103.000000,85103.000000,85103.000000
mean,1.355681e+06,134.872930,0.312774,7.193703,0.543412,3.348366,0.039822,9.585609e+02,159.772570,98.909091,19.859394,7.755176e+02,104.729681,10.680105,93.316029,11.467328
std,6.949952e+05,5403.548851,2.254721,12.362478,13.721223,15.421471,1.791013,2.435920e+04,4574.583903,0.857747,171.446874,1.789338e+04,1142.447517,188.840006,1510.732101,205.372944
min,1.000000e+01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,97.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
25%,7.723900e+05,0.000000,0.000000,0.990000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,98.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
50%,1.331540e+06,0.000000,0.000000,4.490000,0.000000,0.000000,0.000000,7.000000e+00,2.000000,99.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
75%,1.918415e+06,1.000000,0.000000,9.990000,0.000000,0.000000,0.000000,4.500000e+01,14.000000,100.000000,18.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
max,2.765800e+06,872138.000000,21.000000,999.980000,2366.000000,97.000000,100.000000,5.764420e+06,895978.000000,100.000000,9821.000000,3.441592e+06,145727.000000,19159.000000,208473.000000,19159.000000


In [11]:
games.columns

Index(['AppID', 'Name', 'Release date', 'Estimated owners', 'Peak CCU',
       'Required age', 'Price', 'DLC count', 'About the game',
       'Supported languages', 'Full audio languages', 'Reviews',
       'Header image', 'Website', 'Support url', 'Support email', 'Windows',
       'Mac', 'Linux', 'Metacritic score', 'Metacritic url', 'User score',
       'Positive', 'Negative', 'Score rank', 'Achievements', 'Recommendations',
       'Notes', 'Average playtime forever', 'Average playtime two weeks',
       'Median playtime forever', 'Median playtime two weeks', 'Developers',
       'Publishers', 'Categories', 'Genres', 'Tags', 'Screenshots', 'Movies'],
      dtype='object')

# Conclusão